Team 3: Michael Foley, Karen Garcia, Jason Raeppold

In [25]:
#Project 1 - Portfolio Investment Advisor - Alcohol vs Marijuana

#Determine the following:
#-Historical industry performance
#-Industry performance comparison
#-Industry risk analysis
#-Portfolio recommendation

In [26]:
##Begin Pseudocode Here

In [27]:
#Import the required libraries and dependencies
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import hvplot.pandas #Not working in alpaca environment
from MCForecastTools import MCSimulation

%matplotlib inline

In [28]:
# Set Alpaca API key and secret

load_dotenv()
api_key= os.getenv("ALPACA_API_KEY")
sec_key= os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object

alpaca = tradeapi.REST(
    api_key,
    sec_key,
    api_version="v2"
)

In [40]:
# Get 3 years worth of historical data for Alcohol stocks

start_date = pd.Timestamp('2018-04-20', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2021-04-20', tz='America/New_York').isoformat()

tickers = ["BUD", "DEO", "HEINY", "AMBEV", "STZ"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for Alcohol Stocks

df_alc_data = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df
# declare variable name BUD, set value (by using equal) to df_alc_data
# when df_alc_data symbol column is equal to BUD, finally drop symbol column. 
BUD = df_alc_data[df_alc_data["symbol"]=="BUD"].drop("symbol", axis=1) # 1 would be column, 0 would be index. 
DEO = df_alc_data[df_alc_data["symbol"]=="DEO"].drop("symbol", axis=1)
HEINY = df_alc_data[df_alc_data["symbol"]=="HEINY"].drop("symbol", axis=1)
AMBEV = df_alc_data[df_alc_data["symbol"]=="AMBEV"].drop("symbol", axis=1)
STZ = df_alc_data[df_alc_data["symbol"]=="STZ"].drop("symbol", axis=1)
df_alc_data.index = df_alc_data.index.date
df_alc_data.head(-5)


,open,high,low,close,volume,trade_count,vwap,symbol
2018-04-20,105.51,105.530,103.60,103.94,2071048,16933,104.238972,BUD
2018-04-23,103.29,103.545,102.90,103.07,1466616,12473,103.164874,BUD
2018-04-24,103.60,103.650,101.96,102.48,1862054,15938,102.658909,BUD
2018-04-25,102.71,103.530,102.51,103.23,1867165,15315,103.103000,BUD
2018-04-26,104.45,104.630,101.76,101.90,3232211,23499,102.796996,BUD
...,...,...,...,...,...,...,...,...
2021-04-07,236.23,236.410,233.69,234.94,1346876,20375,235.127548,STZ
2021-04-08,222.68,226.765,220.98,224.21,4388218,63010,223.982142,STZ
2021-04-09,223.00,225.320,218.62,224.15,2957872,45286,222.039764,STZ
2021-04-12,224.72,227.370,223.81,226.31,1157996,19538,226.008192,STZ


In [30]:
df_alc_data[df_alc_data["symbol"]=="BUD"]["close"].hvplot()

:Curve   [index]   (close)

In [31]:
# Get 3 years worth of historical data for Cannabis stocks

start_date = pd.Timestamp('2018-04-20', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2021-04-20', tz='America/New_York').isoformat()

tickers = ["CURLF","GTBIF","GWPH", "TLRY","CGC"]
# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for Cannabis Stocks

df_cann_data = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df


CURLF = df_cann_data[df_cann_data["symbol"]=="CURLF"].drop("symbol", axis=1)
GTBIF = df_cann_data[df_cann_data["symbol"]=="GTBIF"].drop("symbol", axis=1)
GWPH = df_cann_data[df_cann_data["symbol"]=="GWPH"].drop("symbol", axis=1)
TLRY = df_cann_data[df_cann_data["symbol"]=="TLRY"].drop("symbol", axis=1)
CGC = df_cann_data[df_cann_data["symbol"]=="CGC"].drop("symbol", axis=1)
df_cann_data.index = df_cann_data.index.date
df_cann_data.head(-5)

,open,high,low,close,volume,trade_count,vwap,symbol
2018-05-24,31.25,31.3100,28.1100,28.2000,2945346,14347,29.486656,CGC
2018-05-25,28.29,28.7399,26.7000,26.9000,2787274,12761,27.310695,CGC
2018-05-29,27.86,29.0000,27.2500,28.1200,2603463,12050,28.172655,CGC
2018-05-30,28.40,28.9000,28.0500,28.8800,2971397,10770,28.559480,CGC
2018-05-31,29.19,29.1900,28.1400,28.3700,1359345,7214,28.573360,CGC
...,...,...,...,...,...,...,...,...
2021-04-07,20.91,20.9800,19.5700,19.7600,18577841,118325,20.084978,TLRY
2021-04-08,19.87,20.1894,19.0800,19.7700,13496556,89303,19.701066,TLRY
2021-04-09,19.83,20.6800,19.4031,19.7600,14034781,84753,20.023209,TLRY
2021-04-12,17.97,18.2100,17.1000,17.1500,27160303,178111,17.452997,TLRY


In [32]:
# Concatenate the ticker DataFrames ### Not sure if this is even necessary, just thought it was odd that there are so many NaN values when you can see the data above.

df_stock_data = pd.concat([BUD,DEO,HEINY,AMBEV,STZ,CURLF,GTBIF,GWPH,TLRY,CGC],axis=1, keys=["BUD", "DEO", "HEINY", "AMBEV", "STZ", "CURLF", "GTBIF", "GWPH", "TLRY", "CGC"])
# Preview DataFrame
df_stock_data.index = df_stock_data.index.date
#df_stock_data

# Display sample data
df_stock_data.head(-5)

BUD                                            DEO            \
           open high low close volume trade_count vwap    open      high   
2018-04-20  NaN  NaN NaN   NaN    NaN         NaN  NaN  138.14  138.3900   
2018-04-23  NaN  NaN NaN   NaN    NaN         NaN  NaN  136.63  137.3600   
2018-04-24  NaN  NaN NaN   NaN    NaN         NaN  NaN  138.38  138.5100   
2018-04-25  NaN  NaN NaN   NaN    NaN         NaN  NaN  137.24  137.8400   
2018-04-26  NaN  NaN NaN   NaN    NaN         NaN  NaN  139.13  139.7100   
...         ...  ...  ..   ...    ...         ...  ...     ...       ...   
2021-04-07  NaN  NaN NaN   NaN    NaN         NaN  NaN  171.47  171.4900   
2021-04-08  NaN  NaN NaN   NaN    NaN         NaN  NaN  174.63  175.4000   
2021-04-09  NaN  NaN NaN   NaN    NaN         NaN  NaN  175.20  175.5787   
2021-04-12  NaN  NaN NaN   NaN    NaN         NaN  NaN  173.14  174.3400   
2021-04-13  NaN  NaN NaN   NaN    NaN         NaN  NaN  172.70  174.1150   

                      ...        TLRY                           CGC           \
                 low  ...      volume trade_count       vwap   open     high   
2018-04-20  137.5450  ...         NaN         NaN        NaN    NaN      NaN   
2018-04-23  136.5000  ...         NaN         NaN        NaN    NaN      NaN   
2018-04-24  136.7910  ...         NaN         NaN        NaN    NaN      NaN   
2018-04-25  136.7700  ...         NaN         NaN        NaN    NaN      NaN   
2018-04-26  138.6800  ...         NaN         NaN        NaN    NaN      NaN   
...              ...  ...         ...         ...        ...    ...      ...   
2021-04-07  169.1500  ...  18577841.0    118325.0  20.084978  30.74  30.9200   
2021-04-08  173.9100  ...  13496556.0     89303.0  19.701066  30.12  30.1938   
2021-04-09  174.0400  ...  14034781.0     84753.0  20.023209  28.50  29.4900   
2021-04-12  173.0500  ...  27160303.0    178111.0  17.452997  28.62  28.6700   
2021-04-13  172.3715  ...  23109364.0    135733.0  17.694360  27.27  28.1700   

                                                              
                low  close     volume trade_count       vwap  
2018-04-20      NaN    NaN        NaN         NaN        NaN  
2018-04-23      NaN    NaN        NaN         NaN        NaN  
2018-04-24      NaN    NaN        NaN         NaN        NaN  
2018-04-25      NaN    NaN        NaN         NaN        NaN  
2018-04-26      NaN    NaN        NaN         NaN        NaN  
...             ...    ...        ...         ...        ...  
2021-04-07  29.6400  29.94  2904252.0     30752.0  30.156397  
2021-04-08  28.2000  28.50  8049784.0     71307.0  28.819127  
2021-04-09  27.8150  28.99  4092327.0     40049.0  28.830225  
2021-04-12  27.3500  27.63  3312290.0     43909.0  27.714650  
2021-04-13  26.9201  27.83  3000201.0     31695.0  27.653363  

[750 rows x 70 columns]

In [33]:
# Create and empty DataFrame for closing prices ##### Trying to get close prices, lots of NaN from above.
df_closing_prices = pd.DataFrame()

# Fetch the closing prices for all the tickers
for ticker in tickers:
    df_closing_prices[ticker] = df_stock_data[ticker]["close"]

# Drop the time component of the date
#df_closing_prices.index = df_closing_prices.index.date

# Display sample data
df_closing_prices.head(-5)

,CURLF,GTBIF,GWPH,TLRY,CGC
2018-04-20,NaN,NaN,132.43,NaN,NaN
2018-04-23,NaN,NaN,137.01,NaN,NaN
2018-04-24,NaN,NaN,135.91,NaN,NaN
2018-04-25,NaN,NaN,138.59,NaN,NaN
2018-04-26,NaN,NaN,136.78,NaN,NaN
...,...,...,...,...,...
2021-04-07,NaN,NaN,217.28,19.7600,29.94
2021-04-08,NaN,NaN,218.00,19.7700,28.50
2021-04-09,NaN,NaN,217.93,19.7600,28.99
2021-04-12,NaN,NaN,217.76,17.1500,27.63


In [34]:
########### This is the API call for ALL TICKERS -- Get 3 years worth of historical data 
###########
start_date = pd.Timestamp('2018-04-20', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2021-04-20', tz='America/New_York').isoformat()

tickers = ["BUD", "DEO", "HEINY", "AMBEV", "STZ", "CURLF","GTBIF","GWPH", "TLRY","CGC"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and AGG

df_stock_data = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

BUD = df_stock_data[df_stock_data["symbol"]=="AGG"].drop("symbol", axis=1)
DEO = df_stock_data[df_stock_data["symbol"]=="DEO"].drop("symbol", axis=1)
HEINY = df_stock_data[df_stock_data["symbol"]=="HEINY"].drop("symbol", axis=1)
AMBEV = df_stock_data[df_stock_data["symbol"]=="AMBEV"].drop("symbol", axis=1)
STZ = df_stock_data[df_stock_data["symbol"]=="STZ"].drop("symbol", axis=1)
CURLF = df_stock_data[df_stock_data["symbol"]=="CURLF"].drop("symbol", axis=1)
GTBIF = df_stock_data[df_stock_data["symbol"]=="GTBIF"].drop("symbol", axis=1)
GWPH = df_stock_data[df_stock_data["symbol"]=="GWPH"].drop("symbol", axis=1)
TLRY = df_stock_data[df_stock_data["symbol"]=="TLRY"].drop("symbol", axis=1)
CGC = df_stock_data[df_stock_data["symbol"]=="CGC"].drop("symbol", axis=1)



# Concatenate the ticker DataFrames

df_stock_data = pd.concat([BUD,DEO,HEINY,AMBEV,STZ,CURLF,GTBIF,GWPH,TLRY,CGC],axis=1, keys=["BUD", "DEO", "HEINY", "AMBEV", "STZ", "CURLF", "GTBIF", "GWPH", "TLRY", "CGC"])
# Preview DataFrame
df_stock_data.index = df_stock_data.index.date
df_stock_data

# Display sample data
df_stock_data.head(-5)

BUD                                            DEO            \
           open high low close volume trade_count vwap    open      high   
2018-04-20  NaN  NaN NaN   NaN    NaN         NaN  NaN  138.14  138.3900   
2018-04-23  NaN  NaN NaN   NaN    NaN         NaN  NaN  136.63  137.3600   
2018-04-24  NaN  NaN NaN   NaN    NaN         NaN  NaN  138.38  138.5100   
2018-04-25  NaN  NaN NaN   NaN    NaN         NaN  NaN  137.24  137.8400   
2018-04-26  NaN  NaN NaN   NaN    NaN         NaN  NaN  139.13  139.7100   
...         ...  ...  ..   ...    ...         ...  ...     ...       ...   
2021-04-07  NaN  NaN NaN   NaN    NaN         NaN  NaN  171.47  171.4900   
2021-04-08  NaN  NaN NaN   NaN    NaN         NaN  NaN  174.63  175.4000   
2021-04-09  NaN  NaN NaN   NaN    NaN         NaN  NaN  175.20  175.5787   
2021-04-12  NaN  NaN NaN   NaN    NaN         NaN  NaN  173.14  174.3400   
2021-04-13  NaN  NaN NaN   NaN    NaN         NaN  NaN  172.70  174.1150   

                      ...        TLRY                           CGC           \
                 low  ...      volume trade_count       vwap   open     high   
2018-04-20  137.5450  ...         NaN         NaN        NaN    NaN      NaN   
2018-04-23  136.5000  ...         NaN         NaN        NaN    NaN      NaN   
2018-04-24  136.7910  ...         NaN         NaN        NaN    NaN      NaN   
2018-04-25  136.7700  ...         NaN         NaN        NaN    NaN      NaN   
2018-04-26  138.6800  ...         NaN         NaN        NaN    NaN      NaN   
...              ...  ...         ...         ...        ...    ...      ...   
2021-04-07  169.1500  ...  18577841.0    118325.0  20.084978  30.74  30.9200   
2021-04-08  173.9100  ...  13496556.0     89303.0  19.701066  30.12  30.1938   
2021-04-09  174.0400  ...  14034781.0     84753.0  20.023209  28.50  29.4900   
2021-04-12  173.0500  ...  27160303.0    178111.0  17.452997  28.62  28.6700   
2021-04-13  172.3715  ...  23109364.0    135733.0  17.694360  27.27  28.1700   

                                                              
                low  close     volume trade_count       vwap  
2018-04-20      NaN    NaN        NaN         NaN        NaN  
2018-04-23      NaN    NaN        NaN         NaN        NaN  
2018-04-24      NaN    NaN        NaN         NaN        NaN  
2018-04-25      NaN    NaN        NaN         NaN        NaN  
2018-04-26      NaN    NaN        NaN         NaN        NaN  
...             ...    ...        ...         ...        ...  
2021-04-07  29.6400  29.94  2904252.0     30752.0  30.156397  
2021-04-08  28.2000  28.50  8049784.0     71307.0  28.819127  
2021-04-09  27.8150  28.99  4092327.0     40049.0  28.830225  
2021-04-12  27.3500  27.63  3312290.0     43909.0  27.714650  
2021-04-13  26.9201  27.83  3000201.0     31695.0  27.653363  

[750 rows x 70 columns]

In [35]:
#Clean data, drop nulls

In [36]:
#Plot data

In [37]:
#

In [38]:
#

In [39]:
#